In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
603104,2022-02-04,30-39
603105,2022-02-04,30-39
603106,2022-02-04,20-29
603107,2022-02-04,10-19_
603108,2022-02-04,30-39


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-02-01,809,1082,1967,1454,948,694,449,243,97,34
2022-02-02,664,758,1428,1128,742,494,307,175,87,26
2022-02-03,584,565,671,786,558,405,281,155,62,24
2022-02-04,473,339,449,546,362,279,206,116,55,24
2022-02-05,468,332,392,531,378,242,185,115,49,10


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

2702

In [7]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [8]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
58,2022-02-01,621,579
59,2022-02-02,809,1082
60,2022-02-03,664,758
61,2022-02-04,584,565
62,2022-02-05,473,339


In [9]:
df_0_19.to_csv("../data/cases_0_19.csv")